1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)



**Прогнозирования смены профессии**

In [1]:
#Загрузим библиотеки и посмотрим на данные
import pandas as pd
import numpy as np

df = pd.read_csv("./aug_train.csv")
df.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


enrollee_id: уникальный идентификатор кандидата

city: Код города

city_development_index: индекс развития города (в масштабе)

gender: Пол кандидата

relvent_experience: Соответствующий опыт кандидата

enrolled_university: Тип зачисленных университетских курсов, если таковые имеются

education_level: Уровень образования кандидата

major_discipline: Обучение основной дисциплине кандидата

experience: Кандидатский общий стаж в годах

company_size: Количество сотрудников в компании текущего работодателя

company_type: Тип текущего работодателя

last_new_job: разница в годах между предыдущей работой и текущей работой

training_hours: завершенные часы обучения

target: 0 - Не ищу смены работы, 1 - Ищу смены работы

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [5]:
df['target'].value_counts()

0.0    14381
1.0     4777
Name: target, dtype: int64

In [20]:
dataDesc= []

for i in df.columns:
    dataDesc.append([
        i, 
        df[i].dtypes,
        df[i].isna().sum(),
        round((((df[i].isna().sum()) / len(df)) * 100),2),
        df[i].nunique(),
        df[i].sample(20).drop_duplicates().values
    ])
pd.DataFrame(dataDesc, columns=[
    "Data features", 
    "Data types",
    "Null",
    "Null Percentage",
    "Unique",
    "Unique Sample"
])

,Data features,Data types,Null,Null Percentage,Unique,Unique Sample
0,enrollee_id,int64,0,0.00,19158,"[4287, 1301, 19434, 12454, 26268, 31382, 25530..."
1,city,object,0,0.00,123,"[city_16, city_75, city_21, city_103, city_136..."
2,city_development_index,float64,0,0.00,93,"[0.802, 0.926, 0.92, 0.624, 0.804, 0.91, 0.939..."
3,gender,object,4508,23.53,3,"[Male, nan, Female]"
4,relevent_experience,object,0,0.00,2,"[Has relevent experience, No relevent experience]"
5,enrolled_university,object,386,2.01,3,"[Full time course, no_enrollment, Part time co..."
6,education_level,object,460,2.40,5,"[Graduate, Phd, High School, Masters]"
7,major_discipline,object,2813,14.68,6,"[nan, STEM, Humanities, Other]"
8,experience,object,65,0.34,22,"[>20, 4, 11, 3, 2, 5, 12, nan, 10, 1, 9, 6]"
9,company_size,object,5938,30.99,8,"[50-99, 100-500, <10, nan, 1000-4999, 5000-999..."


In [21]:
#заполним пробелы
for column in ['gender','enrolled_university','education_level','major_discipline', 'experience', 'company_size', 'company_type','last_new_job']:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [22]:
df.isna().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [23]:
#удалим id
df = df.drop(columns=["enrollee_id"])

In [24]:
categorical = df.select_dtypes('object').columns.to_list()

assoc_cramers = []
for i in  categorical:
    assoc = round(cramers_v(df['target'], df[i]),2)
    assoc_cramers.append(assoc)
    
categorical

NameError: name 'cramers_v' is not defined